# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [ ]:
data = run_engine(data)

  0%|                            | 145950/32760501 [01:35<4:32:54, 1991.84it/s]

pl    0.00543
dtype: float64
--------------------------------------
             DateTime_frmt          close_type       pl
130971 2020-01-03 13:46:44  simple_take_profit  0.00034
131612 2020-01-03 13:54:20         simple_stop -0.00035
132611 2020-01-03 14:05:57  simple_take_profit  0.00026
135115 2020-01-03 14:33:46         simple_stop -0.00033
136429 2020-01-03 14:46:29         simple_stop -0.00034
136752 2020-01-03 14:49:01  simple_take_profit  0.00014
137820 2020-01-03 15:00:00  simple_take_profit  0.00126
139501 2020-01-03 15:07:16  simple_take_profit  0.00017
141001 2020-01-03 15:17:33         simple_stop -0.00032
141774 2020-01-03 15:22:44  simple_take_profit  0.00023
142225 2020-01-03 15:26:50  simple_take_profit  0.00016
142732 2020-01-03 15:31:09  simple_take_profit  0.00016
143606 2020-01-03 15:40:41  simple_take_profit  0.00020
143988 2020-01-03 15:45:23  simple_take_profit  0.00020
144825 2020-01-03 15:53:07  simple_take_profit  0.00044


  0%|▏                           | 146293/32760501 [01:35<5:54:48, 1532.01it/s]


In [ ]:
generate_result_report(data)

In [ ]:
plot_graph(data)